### Convert H.264 Stream to MP4 Container

In [ ]:
# Import required libraries
import os
import subprocess
import shutil
from pathlib import Path
from IPython.display import Video, display
import matplotlib.pyplot as plt
import time

#### Function to convert H.264 to MP4

In [2]:
def h264_to_mp4(input_path, output_path, framerate=30, crf=23, preset='medium', audio=None):
    """
    Parameters:
    - input_path: Path to the input H.264 file
    - output_path: Path for the output MP4 file
    - framerate: Frames per second (default: 30)
    - crf: Constant Rate Factor (0-51, lower is better quality, default: 23)
    - preset: Encoding speed preset (ultrafast, superfast, veryfast, faster, fast, 
              medium, slow, slower, veryslow, default: medium)
    - audio: Optional path to audio file to include in the output
    """
    # Check if input file exists
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"Input file {input_path} does not exist")

    # Base command
    cmd = [
        'ffmpeg',
        '-y',  # Overwrite output file if it exists
        '-r', str(framerate),  # Input framerate
        '-i', input_path,  # Input file
        '-c:v', 'libx264',  # Video codec
        '-crf', str(crf),  # Quality setting
        '-preset', preset,  # Encoding speed preset
        '-pix_fmt', 'yuv420p',  # Pixel format for compatibility
        '-movflags', '+faststart'  # Enable streaming
    ]

    # Add audio if provided
    if audio and os.path.exists(audio):
        cmd.extend(['-i', audio, '-c:a', 'aac', '-strict', 'experimental'])

    # Add output path
    cmd.append(output_path)

    # Run the command
    print("Running command:", ' '.join(cmd))
    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode != 0:
        print("Error during conversion:")
        print(result.stderr)
        return False

    print(f"Successfully converted {input_path} to {output_path}")
    return True

#### Function to get video information

In [ ]:
def get_video_info(file_path):
    """Get information about a video file using ffprobe"""
    cmd = [
        'ffprobe',
        '-v', 'quiet',
        '-print_format', 'json',
        '-show_format',
        '-show_streams',
        file_path
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        print("Error getting video info:")
        print(result.stderr)
        return None

    import json
    return json.loads(result.stdout)

#### Function to display video information

In [3]:
def display_video_info(info):
    """Display video information in a readable format"""
    if not info:
        return

    print("Video Information:")
    print("=" * 50)

    if 'streams' in info:
        video_streams = [s for s in info['streams'] if s['codec_type'] == 'video']
        audio_streams = [s for s in info['streams'] if s['codec_type'] == 'audio']

        for i, stream in enumerate(video_streams):
            print(f"Video Stream {i}:")
            print(f"  Codec: {stream.get('codec_name', 'N/A')}")
            print(f"  Resolution: {stream.get('width', 'N/A')}x{stream.get('height', 'N/A')}")
            print(f"  Bitrate: {stream.get('bit_rate', 'N/A')}")
            print(f"  Duration: {stream.get('duration', 'N/A')} seconds")
            print(f"  Framerate: {stream.get('r_frame_rate', 'N/A')}")
            print()

        for i, stream in enumerate(audio_streams):
            print(f"Audio Stream {i}:")
            print(f"  Codec: {stream.get('codec_name', 'N/A')}")
            print(f"  Sample Rate: {stream.get('sample_rate', 'N/A')} Hz")
            print(f"  Channels: {stream.get('channels', 'N/A')}")
            print(f"  Bitrate: {stream.get('bit_rate', 'N/A')}")
            print()

    if 'format' in info:
        print("Container Format:")
        print(f"  Format: {info['format'].get('format_name', 'N/A')}")
        print(f"  Duration: {info['format'].get('duration', 'N/A')} seconds")
        print(f"  Size: {int(info['format'].get('size', 0)) / (1024*1024):.2f} MB")
        print(f"  Bitrate: {info['format'].get('bit_rate', 'N/A')}")

#### Function to display video in notebook

In [4]:
def show_video(video_path, width=640):
    """Display video in Jupyter notebook"""
    if not os.path.exists(video_path):
        print(f"Video file {video_path} does not exist")
        return

    return Video(video_path, width=width, embed=True)

#### Main conversion function with user interface

In [6]:
def convert_h264_to_mp4():
    """Main function to handle the conversion process"""

    # Get input file path
    input_path = input("Enter the path to your H.264 file: ").strip().strip('"')

    if not os.path.exists(input_path):
        print(f"File {input_path} does not exist")
        return

    # Set output path
    default_output = os.path.splitext(input_path)[0] + ".mp4"
    output_path = input(f"Enter output path (default: {default_output}): ").strip().strip('"')
    if not output_path:
        output_path = default_output

    # Get conversion parameters
    try:
        framerate = int(input("Enter framerate (default: 30): ") or "30")
        crf = int(input("Enter CRF value (0-51, lower is better quality, default: 23): ") or "23")
        preset = input("Enter preset (ultrafast, superfast, veryfast, faster, fast, medium, slow, slower, veryslow, default: medium): ") or "medium"

        # Validate preset
        valid_presets = ['ultrafast', 'superfast', 'veryfast', 'faster', 'fast', 'medium', 'slow', 'slower', 'veryslow']
        if preset not in valid_presets:
            print(f"Invalid preset. Using 'medium' instead.")
            preset = 'medium'

    except ValueError:
        print("Invalid input. Using default values.")
        framerate = 30
        crf = 23
        preset = 'medium'

    # Ask for audio file
    audio_path = input("Enter path to audio file (optional, press Enter to skip): ").strip().strip('"')
    if audio_path and not os.path.exists(audio_path):
        print(f"Audio file {audio_path} does not exist. Proceeding without audio.")
        audio_path = None

    # Confirm conversion
    print("\nConversion settings:")
    print(f"Input: {input_path}")
    print(f"Output: {output_path}")
    print(f"Framerate: {framerate}")
    print(f"CRF: {crf}")
    print(f"Preset: {preset}")
    print(f"Audio: {audio_path if audio_path else 'None'}")

    confirm = input("\nProceed with conversion? (y/n): ").strip().lower()
    if confirm != 'y':
        print("Conversion cancelled.")
        return

    # Perform conversion
    print("Starting conversion...")
    start_time = time.time()

    success = h264_to_mp4(input_path, output_path, framerate, crf, preset, audio_path)

    if success:
        end_time = time.time()
        print(f"Conversion completed in {end_time - start_time:.2f} seconds")

        # Get and display video info
        print("\nOriginal H.264 file info:")
        orig_info = get_video_info(input_path)
        display_video_info(orig_info)

        print("\nConverted MP4 file info:")
        mp4_info = get_video_info(output_path)
        display_video_info(mp4_info)

        # Display the video
        print("\nDisplaying converted video:")
        return show_video(output_path)
    else:
        print("Conversion failed.")

# Run the conversion process
convert_h264_to_mp4()

NameError: name 'os' is not defined